## 개요
여러 체크포인트들을 저장하기 위해서는 dictionary에 체크포인트들을 구성하고 ```torch.save()```를 사용하여 사전을 직렬화(serializer)해야한다. 일반적인 PyTorch에서는 이러한 여러 체크포인트들을 저장할 때 ```.tar``` 확장자를 사용하는 것이 일반적인 규칙이다. 항목들을 불러올 때는, 먼저 모델과 옵티마이저를 초기화하고 torch.load()를 사용하여 dictionary를 불러온다. 이후 원하는대로 저장한 항목들을 조회하여 접근할 수 있다.

## 단계
1. 데이터를 불러올 때 필요한 라이브러리들 불러오기
2. 신경망을 구성하고 초기화하기
3. 옵티마이저 초기화하기
4. 일반적인 체크포인트 저장하기
5. 일반적인 체크포인트 불러오기

### 1. 데이터를 불러올 때 필요한 라이브러리들 불러오기

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

### 2. 신경망을 구성하고 초기화하기

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


### 3. 옵티마이저 초기화하기
모멘텀(momentum)을 갖는 SGD를 사용

In [3]:
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

### 4. 일반적인 체크포인트 저장하기
관련된 모든 정보들을 모아서 사전을 구성한다.

In [4]:
# 추가 정보
EPOCH = 5
PATH = "model.pt"
LOSS = 0.4

torch.save({
    'epoch':EPOCH,
    'model_state_dict':net.state_dict(),
    'optimizer_state_dict':optimizer.state_dict(),
    'loss':LOSS,
    }, PATH)

### 5. 일반적인 체크포인트 불러오기
먼저 모델과 옵티마이저를 초기화한 뒤, dictionary를 불러온다.

In [5]:
model = Net()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
# - 또는 -
model.train()

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

추론을 실행하기 전에 model.eval()을 호출하여 드롭아웃(dropout)과 배치 정규화 층을 evaluation 모드로 바꿔야한다. 이것을 빼먹으면 일관성 없는 추론 결과를 얻게 된다. <br>
만약 학습을 계속하길 원한다면 ```model.train()```을 호출하여 이 층(layer)들이 학습 모드인지 ensure해야한다.